<a href="https://colab.research.google.com/github/kraven681/Wallet-Risk-Scoring-From-Scratch/blob/main/Wallet_Risk_Scoring_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as ans

In [ ]:
df = pd.read_csv('/content/Wallet Risk Scoring From Scratch .csv')

In [ ]:
df.head()

,wallet_id
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,0x06b51c6882b27cb05e712185531c1f74996dd988
2,0x0795732aacc448030ef374374eaae57d2965c16c
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   wallet_id  103 non-null    object
dtypes: object(1)
memory usage: 956.0+ bytes


In [ ]:
df.describe()

,wallet_id
count,103
unique,103
top,0x0039f22efb07a647557c7c5d17854cfd6d489ef3
freq,1


In [ ]:
df.dtypes

,0
wallet_id,object


In [ ]:
df.columns

Index(['wallet_id'], dtype='object')

In [ ]:
df.isnull().sum()

,0
wallet_id,0


In [ ]:
import requests
import pandas as pd
import time

# Load wallet IDs (replace with your list if needed)
wallets = [
    "0x0039f22efb07a647557c7c5d17854cfd6d489ef3",
    "0x06b51c6882b27cb05e712185531c1f74996dd988",
    # Add the remaining wallets (up to 100)
]

# GraphQL endpoint for Compound V2
url = "https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2"

# Function to fetch account data from the subgraph
def fetch_wallet_data(wallet):
    query = """
    {
      account(id: "%s") {
        id
        tokens {
          symbol
          supplyBalanceUnderlying
          borrowBalanceUnderlying
        }
        countLiquidated
        countLiquidator
      }
    }
    """ % wallet.lower()

    response = requests.post(url, json={'query': query})
    if response.status_code != 200:
        print(f"Failed for {wallet}")
        return None

    data = response.json().get("data", {}).get("account")
    if not data:
        return {
            "wallet_id": wallet,
            "total_supplied": 0,
            "total_borrowed": 0,
            "num_liquidations": 0,
            "num_times_liquidator": 0
        }

    total_supplied = 0
    total_borrowed = 0
    for token in data["tokens"]:
        total_supplied += float(token.get("supplyBalanceUnderlying", 0))
        total_borrowed += float(token.get("borrowBalanceUnderlying", 0))

    return {
        "wallet_id": wallet,
        "total_supplied": total_supplied,
        "total_borrowed": total_borrowed,
        "net_position": total_supplied - total_borrowed,
        "num_liquidations": int(data.get("countLiquidated", 0)),
        "num_times_liquidator": int(data.get("countLiquidator", 0))
    }

# Process all wallets
results = []
for i, wallet in enumerate(wallets):
    print(f"Fetching {i+1}/{len(wallets)}: {wallet}")
    res = fetch_wallet_data(wallet)
    if res:
        results.append(res)
    time.sleep(0.5)  # Be polite to the API (rate limiting)

# Save as DataFrame
df = pd.DataFrame(results)
df.to_csv("compound_wallet_features.csv", index=False)
print("Saved: compound_wallet_features.csv")



Fetching 1/2: 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
Fetching 2/2: 0x06b51c6882b27cb05e712185531c1f74996dd988
Saved: compound_wallet_features.csv


In [ ]:
import pandas as pd
import requests
import time

# ✅ Load your wallet list
wallets = pd.read_csv("/content/Wallet Risk Scoring From Scratch .csv")["wallet_id"].dropna().unique().tolist()

# ✅ Compound V2 Subgraph endpoint
SUBGRAPH_URL = "https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2"

# ✅ Function to fetch wallet features
def fetch_compound_features(wallet):
    query = f"""
    {{
      account(id: "{wallet.lower()}") {{
        id
        tokens {{
          symbol
          supplyBalanceUnderlying
          borrowBalanceUnderlying
        }}
        countLiquidated
        countLiquidator
      }}
    }}
    """
    try:
        res = requests.post(SUBGRAPH_URL, json={"query": query})
        res.raise_for_status()
        data = res.json()["data"]["account"]
        if data is None:
            return {
                "wallet_id": wallet,
                "total_supplied": 0,
                "total_borrowed": 0,
                "net_position": 0,
                "num_liquidations": 0,
                "num_times_liquidator": 0,
            }

        supplied = sum(float(t.get("supplyBalanceUnderlying", 0)) for t in data.get("tokens", []))
        borrowed = sum(float(t.get("borrowBalanceUnderlying", 0)) for t in data.get("tokens", []))
        return {
            "wallet_id": wallet,
            "total_supplied": supplied,
            "total_borrowed": borrowed,
            "net_position": supplied - borrowed,
            "num_liquidations": int(data.get("countLiquidated", 0)),
            "num_times_liquidator": int(data.get("countLiquidator", 0)),
        }
    except Exception as e:
        print(f"Failed for {wallet[:10]}...: {str(e)}")
        return None

# ✅ Loop through all wallets
features = []
for i, wallet in enumerate(wallets):
    print(f"[{i+1}/{len(wallets)}] Processing {wallet}")
    data = fetch_compound_features(wallet)
    if data:
        features.append(data)
    time.sleep(0.4)  # be kind to the API

# ✅ Save the results
df = pd.DataFrame(features)
df.to_csv("compound_wallet_features.csv", index=False)
print("Saved to compound_wallet_features.csv")


[1/103] Processing 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
Failed for 0x0039f22e...: 'data'
[2/103] Processing 0x06b51c6882b27cb05e712185531c1f74996dd988
Failed for 0x06b51c68...: 'data'
[3/103] Processing 0x0795732aacc448030ef374374eaae57d2965c16c
Failed for 0x0795732a...: 'data'
[4/103] Processing 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
Failed for 0x0aaa79f1...: 'data'
[5/103] Processing 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
Failed for 0x0fe383e5...: 'data'
[6/103] Processing 0x104ae61d8d487ad689969a17807ddc338b445416
Failed for 0x104ae61d...: 'data'
[7/103] Processing 0x111c7208a7e2af345d36b6d4aace8740d61a3078
Failed for 0x111c7208...: 'data'
[8/103] Processing 0x124853fecb522c57d9bd5c21231058696ca6d596
Failed for 0x124853fe...: 'data'
[9/103] Processing 0x13b1c8b0e696aff8b4fee742119b549b605f3cbc
Failed for 0x13b1c8b0...: 'data'
[10/103] Processing 0x1656f1886c5ab634ac19568cd571bc72f385fdf7
Failed for 0x1656f188...: 'data'
[11/103] Processing 0x1724e16cb8d0e2aa4d08035bc6b

In [ ]:
import requests
import json

# Set the GraphQL endpoint for Compound V2
url = "https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2"

# Wallet address
wallet = "0xfaa0768bde629806739c3a4620656c5d26f44ef2"

# Construct the GraphQL query
query = """
{
  account(id: "%s") {
    id
    tokens {
      symbol
      supplyBalanceUnderlying
      borrowBalanceUnderlying
    }
    countLiquidated
    countLiquidator
  }
}
""" % wallet.lower()

# Send POST request
response = requests.post(url, json={'query': query})

# Check response
if response.status_code == 200:
    data = response.json()

    # Check for GraphQL errors
    if "errors" in data:
        print("GraphQL error:", data["errors"])
    elif data.get("data", {}).get("account") is None:
        print(f"No account data found for wallet: {wallet}")
    else:
        print(json.dumps(data["data"]["account"], indent=2))
else:
    print("Query failed with status code", response.status_code)


GraphQL error: [{'message': 'This endpoint has been removed. If you have any questions, reach out to support@thegraph.zendesk.com'}]


In [ ]:
for i, row in df.iterrows():
    total_supplied = row["total_supplied"]
    total_borrowed = row["total_borrowed"]
    net_position = row["net_position"]
    num_liquidations = row["num_liquidations"]
    num_times_liquidator = row["num_times_liquidator"]

    # Calculate utilization rate
    utilization_rate = total_borrowed / total_supplied if total_supplied > 0 else 0

    # Scoring
    score = 1000

    if utilization_rate > 0.8:
        score -= 300

    score -= 200 * num_liquidations

    if net_position < 0:
        score -= 100

    score += 100 * num_times_liquidator

    # Clamp score between 0 and 1000
    score = max(0, min(1000, score))

    # Assign to DataFrame
    df.loc[i, "score"] = score




In [ ]:
print(df.columns)


RangeIndex(start=0, stop=0, step=1)


In [ ]:
{
  "id": "0xfaa076...",
  "tokens": [
    {
      "symbol": "cUSDC",
      "supplyBalanceUnderlying": "54.13",
      "borrowBalanceUnderlying": "0.00"
    }
  ],
  "countLiquidated": 0,
  "countLiquidator": 1
}


{'id': '0xfaa076...',
 'tokens': [{'symbol': 'cUSDC',
   'supplyBalanceUnderlying': '54.13',
   'borrowBalanceUnderlying': '0.00'}],
 'countLiquidated': 0,
 'countLiquidator': 1}

In [ ]:
print(f"{wallet}: {data}")


0xfaa0768bde629806739c3a4620656c5d26f44ef2: {'errors': [{'message': 'This endpoint has been removed. If you have any questions, reach out to support@thegraph.zendesk.com'}]}


In [ ]:
score = 1000  # Assume low risk if wallet has no activity


In [ ]:
print(response.text)


{
  "errors": [
    {
      "message": "This endpoint has been removed. If you have any questions, reach out to support@thegraph.zendesk.com"
    }
  ]
}



In [ ]:
time.sleep(1)


In [ ]:
for i, row in df.iterrows():
    score = 1000
    ...
    row["score"] = score  # ❌ This doesn’t update the dataframe


In [ ]:
df.loc[i, "score"] = score


In [ ]:
import pandas as pd
import requests
import time
from tqdm import tqdm

url = "https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2"

# Read wallet list
wallets = pd.read_csv("/content/Wallet Risk Scoring From Scratch .csv")
wallets.columns = wallets.columns.str.strip()  # Clean column names
results = []

def fetch_wallet_data(wallet):
    query = """
    {
      account(id: "%s") {
        id
        tokens {
          symbol
          supplyBalanceUnderlying
          borrowBalanceUnderlying
        }
        countLiquidated
        countLiquidator
      }
    }
    """ % wallet.lower()

    response = requests.post(url, json={"query": query})
    if response.status_code != 200:
        print(f"Failed request for {wallet}: {response.status_code}")
        return None

    data = response.json()
    if "errors" in data or data.get("data", {}).get("account") is None:
        print(f"No account data for {wallet}")
        return None

    return data["data"]["account"]

def compute_score(wallet_data):
    total_supplied = 0
    total_borrowed = 0
    tokens = wallet_data.get("tokens", [])

    for token in tokens:
        total_supplied += float(token.get("supplyBalanceUnderlying", 0))
        total_borrowed += float(token.get("borrowBalanceUnderlying", 0))

    net_position = total_supplied - total_borrowed
    utilization_rate = total_borrowed / total_supplied if total_supplied > 0 else 0
    num_liquidations = int(wallet_data.get("countLiquidated", 0))
    num_times_liquidator = int(wallet_data.get("countLiquidator", 0))

    # Scoring logic
    score = 1000
    if utilization_rate > 0.8:
        score -= 300
    score -= 200 * num_liquidations
    if net_position < 0:
        score -= 100
    score += 100 * num_times_liquidator
    score = max(0, min(1000, score))  # Clamp

    return score

# Process wallets
for wallet in tqdm(wallets['wallet_id']):
    try:
        data = fetch_wallet_data(wallet)
        if data is None:
            score = 500  # Neutral if no data
        else:
            score = compute_score(data)
        print(f"{wallet}: {score}")
        results.append({'wallet_id': wallet, 'score': score})
        time.sleep(1)  # Slower to avoid rate limits
    except Exception as e:
        print(f"Error processing {wallet}: {e}")
        results.append({'wallet_id': wallet, 'score': 0})

df_scores = pd.DataFrame(results)
df_scores.to_csv("wallet_risk_scores.csv", index=False)
print("✅ Done: Scores saved to wallet_risk_scores.csv")


  0%|          | 0/103 [00:00<?, ?it/s]

No account data for 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
0x0039f22efb07a647557c7c5d17854cfd6d489ef3: 500


  1%|          | 1/103 [00:01<01:51,  1.09s/it]

No account data for 0x06b51c6882b27cb05e712185531c1f74996dd988
0x06b51c6882b27cb05e712185531c1f74996dd988: 500


  2%|▏         | 2/103 [00:02<01:48,  1.08s/it]

...

No account data for 0xf80a8b9cfff0febf49914c269fb8aead4a22f847
0xf80a8b9cfff0febf49914c269fb8aead4a22f847: 500


 99%|█████████▉| 102/103 [01:49<00:01,  1.08s/it]

No account data for 0xfe5a05c0f8b24fca15a7306f6a4ebb7dcf2186ac
0xfe5a05c0f8b24fca15a7306f6a4ebb7dcf2186ac: 500


100%|██████████| 103/103 [01:51<00:00,  1.08s/it]

✅ Done: Scores saved to wallet_risk_scores.csv
